In [1]:
import json
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 파일 경로
file_path = './lastest_one_letter_all.json'

# koelectra tokenizer 불러오기
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# device 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# 클래스 생성
class KoNERDataset(Dataset):
    
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

In [3]:
# ner data 불러오기
def load_ner_data(file_path):
    """JSON 파일에서 NER 데이터 로드"""
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

In [4]:
def prepare_koelectra_ner_data(data, tokenizer, max_length=64):
    """KoElectra용 NER 데이터 준비"""
    tokens = [d['tokens'] for d in data]
    labels = [d['ner_tags'] for d in data]

    # KoElectra 토크나이저로 인코딩
    encodings = tokenizer(
        tokens, 
        is_split_into_words=True, 
        padding=True, 
        truncation=True, 
        max_length=max_length,
        return_tensors='pt'
    )

    # NER 라벨 처리
    new_labels = []
    for i, label in enumerate(labels):
        word_ids = encodings.word_ids(batch_index=i)
        label_ids = [-100] * len(word_ids)  # 초기값 -100으로 설정
        
        # 개선된 라벨 매핑 로직
        for word_idx, tag in enumerate(label):
            # word_ids에서 해당 word_idx의 첫 번째 인덱스 찾기
            indices = [j for j, w_id in enumerate(word_ids) if w_id == word_idx]
            
            if indices:
                # 첫 번째 인덱스에 태그 할당
                label_ids[indices[0]] = tag
        
        new_labels.append(label_ids)

    return encodings, new_labels

- - - 

디버깅

In [5]:
def debug_tokenization(tokens, tokenizer, encodings, new_labels):
    """토크나이징 과정 디버깅"""
    print("원본 토큰:", tokens)
    
    # 단어 ID 출력
    print("Word IDs:", encodings.word_ids(batch_index=0))
    
    # 디코딩된 토큰 확인
    decoded_tokens = tokenizer.convert_ids_to_tokens(encodings['input_ids'][0])
    print("디코딩된 토큰:", decoded_tokens)

    print(f"new_labels : ", new_labels)

- - -

학습

In [6]:
def train_koelectra_ner_model(train_dataset, num_labels=7, epochs=10):
    """KoElectra NER 모델 학습"""
    # KoElectra 모델 초기화
    model = AutoModelForTokenClassification.from_pretrained(
        "monologg/koelectra-base-v3-discriminator", 
        num_labels=num_labels)

    # 학습
    model.to(device)
    model.train()

    # optim 설정 
    optimizer = torch.optim.AdamW(model.parameters(), weight_decay=0.01, lr=1e-3)

    best_loss = 1

    # 학습 루프
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_dataset:
            optimizer.zero_grad()
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids, 
                attention_mask=attention_mask, 
                labels=labels
            )
            
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            
            out_loss = round(total_loss/len(train_dataset), 4)

            if best_loss > out_loss:
                best_loss = out_loss

                model.save_pretrained('./model_oneletter/AdamW_WD2_lr3_model')
                tokenizer.save_pretrained('./model_oneletter/AdamW_WD2_lr3_model')


        print(f"Epoch {epoch+1}/{epochs}, Average Loss: {total_loss/len(train_dataset):.4f}")

    return model

In [7]:
def evaluate_koelectra_ner_model(model, val_dataset):
    """모델 평가"""
    model.to(device)
    model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in val_dataset:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels']

            outputs = model(
                input_ids, 
                attention_mask=attention_mask
            )
            
            predictions = torch.argmax(outputs.logits, dim=-1)
            
            # 유효한 예측과 레이블만 수집
            for pred, label in zip(predictions, labels):
                mask = label != -100
                all_preds.extend(pred[mask].cpu().numpy())
                all_labels.extend(label[mask].numpy())

    # 상세 분류 보고서 출력
    print(classification_report(all_labels, all_preds))

- - -

실행

In [8]:
def main():
    # 데이터 로드
    data = load_ner_data(file_path)

    # 데이터 분할 (학습:검증 = 8:2)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
    train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

    # 데이터 준비
    train_encodings, train_labels = prepare_koelectra_ner_data(train_data, tokenizer)
    val_encodings, val_labels = prepare_koelectra_ner_data(val_data, tokenizer)

    # 토큰화 결과 확인
    debug_tokenization(train_data[0]['tokens'], tokenizer, train_encodings, train_labels)


    # 데이터셋 및 데이터로더 생성
    train_dataset = DataLoader(
        KoNERDataset(train_encodings, train_labels), 
        batch_size=8, 
        shuffle=True
    )
    val_dataset = DataLoader(
        KoNERDataset(val_encodings, val_labels), 
        batch_size=8, 
        shuffle=False
    )

    # 모델 학습
    model = train_koelectra_ner_model(train_dataset)

    # 모델 평가
    evaluate_koelectra_ner_model(model, val_dataset)
    
if __name__ == '__main__':
    main()

원본 토큰: ['강', '남', '면', '옥', '에', '서', '물', '냉', '면', '2', '그', '릇', '과', '비', '빔', '냉', '면', '1', '그', '릇', '부', '탁', '드', '립', '니', '다', '.']
Word IDs: [None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
디코딩된 토큰: ['[CLS]', '강', '남', '면', '옥', '에', '서', '물', '냉', '면', '2', '그', '릇', '과', '비', '빔', '냉', '면', '1', '그', '릇', '부', '탁', '드', '립', '니', '다', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
new_labels :  [[-100, 1, 2, 2, 2, 0, 0, 3, 4, 4,

Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\KDP-23\AppData\Local\Temp\ipykernel_476\2591246099.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch 1/10, Average Loss: 1.6962
Epoch 2/10, Average Loss: 1.6795
Epoch 3/10, Average Loss: 1.6778


KeyboardInterrupt: 

- - -

- - -

test

In [ ]:
def prepare_koelectra_ner_data(data, tokenizer, max_length=64):
   
    """KoElectra용 NER 데이터 준비"""

    # text = []
    tokens = [d['tokens'] for d in data]
    labels = [d['ner_tags'] for d in data]

    # tokens

    # KoElectra 토크나이저로 인코딩
    encodings = tokenizer(
        tokens, 
        is_split_into_words=True, 
        padding=True, 
        truncation=True, 
        max_length=max_length,
        return_tensors='pt'
    )

    # NER 라벨 처리
    new_labels = []
    for i, label in enumerate(labels):
        word_ids = encodings.word_ids(batch_index=i)
        label_ids = [-100] * len(word_ids)  # 초기값 -100으로 설정
        
        # 개선된 라벨 매핑 로직
        for word_idx, tag in enumerate(label):
            # word_ids에서 해당 word_idx의 첫 번째 인덱스 찾기
            indices = [j for j, w_id in enumerate(word_ids) if w_id == word_idx]
            
            if indices:
                # 첫 번째 인덱스에 태그 할당
                label_ids[indices[0]] = tag
        
        new_labels.append(label_ids)

    return encodings, new_labels

In [ ]:
def test_koelectra_ner_model(model, test_dataset, test_data):
    """모델 테스트"""
    model.to(device)
    model.eval()

    all_test_data = []
    all_preds = []
    all_labels = []
    all_tokens = []  # 원본 토큰 저장용
    for a in test_data:
        all_test_data.append(a['tokens'])
    
    with torch.no_grad():
        for batch in test_dataset:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels']

            outputs = model(
                input_ids, 
                attention_mask=attention_mask
            )
            
            predictions = torch.argmax(outputs.logits, dim=-1)
        

            # 유효한 예측과 레이블만 수집
            for pred, label, input_id in zip(predictions, labels, input_ids):
                mask = label != -100
                all_preds.extend(pred[mask].cpu().numpy())
                all_labels.extend(label[mask].numpy())
                 
                # 토큰 디코딩 (원본 토큰 복원)
                tokens = tokenizer.convert_ids_to_tokens(input_id)
                all_tokens.extend([token for token, m in zip(tokens, mask) if m])       
    
    # 상세 분류 보고서 출력
    print("Classification Report:")
    print(classification_report(all_labels, all_preds))

    # 일부 잘못 예측된 케이스 출력 (선택사항)
    print("\n일부 잘못 예측된 예시:")
    for token, true_label, pred_label in zip(all_tokens, all_labels, all_preds):
        if true_label != pred_label:
            print(f"Token: {token}, True Label: {true_label}, Predicted Label: {pred_label}")

In [ ]:
test_file_path = './data/test/test_data_all.json'

def main_test():

    model= AutoModelForTokenClassification.from_pretrained('./model_oneletter/AdamW_WD2_lr3_model')

    # 테스트 데이터 로드 
    test_data = load_ner_data(test_file_path)

    # 테스트 데이터 준비
    test_encodings, test_labels = prepare_koelectra_ner_data(test_data, tokenizer)

    # 테스트 데이터셋 생성
    test_dataset = DataLoader(
        KoNERDataset(test_encodings, test_labels), 
        batch_size=8, 
        shuffle=False
    )

    # 모델 테스트
    test_koelectra_ner_model(model, test_dataset, test_data)

if __name__ == '__main__':
    main_test()

C:\Users\KDP-23\AppData\Local\Temp\ipykernel_8064\2591246099.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Classification Report:
              precision    recall  f1-score   support

           0       0.24      1.00      0.39       578
           1       0.00      0.00      0.00        82
           2       0.00      0.00      0.00       389
           3       0.00      0.00      0.00       165
           4       0.00      0.00      0.00       854
           5       0.00      0.00      0.00       152
           6       0.00      0.00      0.00       174

    accuracy                           0.24      2394
   macro avg       0.03      0.14      0.06      2394
weighted avg       0.06      0.24      0.09      2394


일부 잘못 예측된 예시:
Token: 치, True Label: 1, Predicted Label: 0
Token: 밥, True Label: 2, Predicted Label: 0
Token: 의, True Label: 2, Predicted Label: 0
Token: 달, True Label: 2, Predicted Label: 0
Token: 인, True Label: 2, Predicted Label: 0
Token: 불, True Label: 3, Predicted Label: 0
Token: 향, True Label: 4, Predicted Label: 0
Token: 바, True Label: 4, Predicted Label: 0
Token: 베, Tru

c:\Users\KDP-23\anaconda3\envs\imc_env\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\KDP-23\anaconda3\envs\imc_env\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\KDP-23\anaconda3\envs\imc_env\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# # 저장된 토크나이저 로드
# tokenizer = AutoTokenizer.from_pretrained('./MODEL/koelectra_v3_ner_model2')

# 입력 문장
# sentence = "중화루에서 깐풍기 소자 하나랑 나홀로 세트 b번 두개, 야끼우동 세개 주문할게요"

# # 토큰화
# encoding = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
# input_ids = encoding['input_ids']
# attention_mask = encoding['attention_mask']

In [ ]:
# # 한글자씩 임의로 토큰화
# sentence = "중화루에서 깐풍기 소자 하나랑 나홀로 세트 b번 두개, 야끼우동 세개 주문할게요"
# def new_tokenizer(sentence):
#     text = []
#     a = list(sentence)
#     for b in a:
#         if b == ' ':
#             pass
#         else:
#             text.append(b)
#     return text

# tokenizer = new_tokenizer(sentence)
# print(tokenizer)


모델 불러와 적용

- - -

Claude
- - - 

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('./koelectra_v3_ner_model5')

# 입력 문장
sentence = '백로식당에서 갈비찜 두개랑 비빔밥 하나시켜주세요.'

# 토큰화 및 인코딩 (특별 토큰 포함)
encoding = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

# 모델 로드
model = AutoModelForTokenClassification.from_pretrained('./koelectra_v3_ner_model5')
model.eval()

# 추론
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)  # 가장 높은 확률을 가진 클래스 선택

# 결과 디코딩
decoded_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

# 라벨 매핑 확인 및 출력
print("Available labels:", model.config.label2id)
print("Predictions:", predictions)

# 라벨 매핑 방식 수정
tags = []
for pred in predictions[0]:
    pred_value = pred.item()  # tensor를 정수로 변환
    try:
        tag = list(model.config.label2id.keys())[list(model.config.label2id.values()).index(pred_value)]
        tags.append(tag)
    except ValueError:
        tags.append('O')  # 매칭되는 라벨이 없으면 'O'로 처리

# 결과 출력
for token, tag in zip(decoded_tokens, tags):
    print(f"{token}: {tag}")

- - - 